# <center> cs231n. [A2, part 5](http://cs231n.github.io/assignments2018/assignment2/). Training a ConvNet PyTorch
<img src='../img/cs231n_logo.png' width=70%>
    
#### <center> **Solution by @yorko**

In this notebook, you'll learn how to use the powerful PyTorch framework to specify a conv net architecture and train it on the CIFAR-10 dataset. 

In [1]:
import timeit
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

We are working with PyTorch version 1.0.0. The code will be slightly updated accordingly. 

In [2]:
torch.__version__

'1.0.0'

## What's this PyTorch business?

You've written a lot of code in this assignment to provide a whole host of neural network functionality. Dropout, Batch Norm, and 2D convolutions are some of the workhorses of deep learning in computer vision. You've also worked hard to make your code efficient and vectorized.

For the last part of this assignment, though, we're going to leave behind your beautiful codebase and instead migrate to one of two popular deep learning frameworks: in this instance, PyTorch (or TensorFlow, if you switch over to that notebook). 

Why?

* Our code will now run on GPUs! Much faster training. When using a framework like PyTorch or TensorFlow you can harness the power of the GPU for your own custom neural network architectures without having to write CUDA code directly (which is beyond the scope of this class).
* We want you to be ready to use one of these frameworks for your project so you can experiment more efficiently than if you were writing every feature you want to use by hand. 
* We want you to stand on the shoulders of giants! TensorFlow and PyTorch are both excellent frameworks that will make your lives a lot easier, and now that you understand their guts, you are free to use them :) 
* We want you to be exposed to the sort of deep learning code you might run into in academia or industry. 

## How will I learn PyTorch?

If you've used Torch before, but are new to PyTorch, this tutorial might be of use: http://pytorch.org/tutorials/beginner/former_torchies_tutorial.html

Otherwise, this notebook will walk you through much of what you need to do to train models in Torch. See the end of the notebook for some links to helpful tutorials if you want to learn more or need further clarification on topics that aren't fully explained here.

## Load Datasets

We load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In [3]:
PATH_TO_CIFAR = '/home/kashn500/data/cifar-10-batches-py/' 

In [4]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

NUM_TRAIN = 49000
NUM_VAL = 1000

cifar10_train = dset.CIFAR10(PATH_TO_CIFAR, train=True, download=False,
                           transform=T.ToTensor())
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, 0))

cifar10_val = dset.CIFAR10(PATH_TO_CIFAR, train=True, download=False,
                           transform=T.ToTensor())
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

cifar10_test = dset.CIFAR10(PATH_TO_CIFAR, train=False, download=False,
                          transform=T.ToTensor())
loader_test = DataLoader(cifar10_test, batch_size=64)


For now, we're going to use a CPU-friendly datatype. Later, we'll switch to a datatype that will move all our computations to the GPU and measure the speedup.

In [5]:
dtype = torch.FloatTensor # the CPU datatype

# Constant to control how frequently we print train loss
print_every = 100

# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

## Example Model

### Some assorted tidbits

Let's start by looking at a simple model. First, note that PyTorch operates on Tensors, which are n-dimensional arrays functionally analogous to numpy's ndarrays, with the additional feature that they can be used for computations on GPUs.

We'll provide you with a Flatten function, which we explain here. Remember that our image data (and more relevantly, our intermediate feature maps) are initially N x C x H x W, where:
* N is the number of datapoints
* C is the number of channels
* H is the height of the intermediate feature map in pixels
* W is the height of the intermediate feature map in pixels

This is the right way to represent the data when we are doing something like a 2D convolution, that needs spatial understanding of where the intermediate features are relative to each other. When we input  data into fully connected affine layers, however, we want each datapoint to be represented by a single vector -- it's no longer useful to segregate the different channels, rows, and columns of the data. So, we use a "Flatten" operation to collapse the C x H x W values per representation into a single long vector. The Flatten function below first reads in the N, C, H, and W values from a given batch of data, and then returns a "view" of that data. "View" is analogous to numpy's "reshape" method: it reshapes x's dimensions to be N x ??, where ?? is allowed to be anything (in this case, it will be C x H x W, but we don't need to specify that explicitly). 

In [6]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

### The example model itself

The first step to training your own model is defining its architecture.

Here's an example of a convolutional neural network defined in PyTorch -- try to understand what each line is doing, remembering that each layer is composed upon the previous layer. We haven't trained anything yet - that'll come next - for now, we want you to understand how everything gets set up.  nn.Sequential is a container which applies each layer
one after the other.

In that example, you see 2D convolutional layers (Conv2d), ReLU activations, and fully-connected layers (Linear). You also see the Cross-Entropy loss function, and the Adam optimizer being used. 

Make sure you understand why the parameters of the Linear layer are 5408 and 10.


In [7]:
# Here's where we define the architecture of the model... 
simple_model = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                Flatten(), # see above for explanation
                nn.Linear(5408, 10), # affine layer
              )

# Set the type of all data in this model to be FloatTensor 
simple_model.type(dtype)

loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(simple_model.parameters(), lr=1e-2) # lr sets the learning rate of the optimizer

In [8]:
simple_model

Sequential(
  (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2))
  (1): ReLU(inplace)
  (2): Flatten()
  (3): Linear(in_features=5408, out_features=10, bias=True)
)

PyTorch supports many other layer types, loss functions, and optimizers - you will experiment with these next. Here's the official API documentation for these (if any of the parameters used above were unclear, this resource will also be helpful). One note: what we call in the class "spatial batch norm" is called "BatchNorm2D" in PyTorch.

* Layers: http://pytorch.org/docs/nn.html
* Activations: http://pytorch.org/docs/nn.html#non-linear-activations
* Loss functions: http://pytorch.org/docs/nn.html#loss-functions
* Optimizers: http://pytorch.org/docs/optim.html#algorithms

## Training a specific model

In this section, we're going to specify a model for you to construct. The goal here isn't to get good performance (that'll be next), but instead to get comfortable with understanding the PyTorch documentation and configuring your own model. 

Using the code provided above as guidance, and using the following PyTorch documentation, specify a model with the following architecture:

* 7x7 Convolutional Layer with 32 filters and stride of 1
* ReLU Activation Layer
* Spatial Batch Normalization Layer
* 2x2 Max Pooling layer with a stride of 2
* Affine layer with 1024 output units
* ReLU Activation Layer
* Affine layer from 1024 input units to 10 outputs

And finally, set up a **cross-entropy** loss function and the **RMSprop** learning rule.

In [9]:
fixed_model_base = nn.Sequential( 
    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=7, stride=1),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(num_features=32), # 32 channels, see docs for nn.BatchNorm2d
    nn.MaxPool2d(kernel_size=2, stride=2),
    Flatten(), # see above for explanation
    nn.Linear(in_features=5408, out_features=1024), # affine layer
    nn.ReLU(inplace=True),
    nn.Linear(in_features=1024, out_features=10) # affine layer
            )

fixed_model = fixed_model_base.type(dtype)

To make sure you're doing the right thing, use the following tool to check the dimensionality of your output (it should be 64 x 10, since our batches have size 64 and the output of the final affine layer should be 10, corresponding to our 10 classes):

In [10]:
## Now we're going to feed a random batch into the model you defined and make sure the output is the right size
x = torch.randn(64, 3, 32, 32).type(dtype)
ans = fixed_model(x)        # Feed it through the model! 

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
np.array_equal(np.array(ans.size()), np.array([64, 10]))       

True

### GPU!

Now, we're going to switch the dtype of the model and our data to the GPU-friendly tensors, and see what happens... everything is the same, except we are casting our model and input tensors as this new dtype instead of the old one.

If this returns false, or otherwise fails in a not-graceful way (i.e., with some error message), you may not have an NVIDIA GPU available on your machine. If you're running locally, we recommend you switch to Google Cloud and follow the instructions to set up a GPU there. If you're already on Google Cloud, something is wrong -- make sure you followed the instructions on how to request and use a GPU on your instance. If you did, post on Piazza or come to Office Hours so we can help you debug.

In [11]:
# Verify that CUDA is properly configured and you have a GPU available

torch.cuda.is_available()

True

In [12]:
!nvidia-smi

Tue Feb 19 22:59:03 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:03:00.0  On |                  N/A |
| 45%   59C    P8    14W / 180W |     85MiB /  8116MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro P6000        Off  | 00000000:04:00.0 Off |                  Off |
| 26%   32C    P8    17W / 250W |     12MiB / 24449MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [13]:
import copy
gpu_dtype = torch.cuda.FloatTensor

fixed_model_gpu = copy.deepcopy(fixed_model_base).type(gpu_dtype)

x_gpu = torch.randn(64, 3, 32, 32).type(gpu_dtype)
ans = fixed_model_gpu(x.type(gpu_dtype))        # Feed it through the model! 

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
np.array_equal(np.array(ans.size()), np.array([64, 10]))

True

Run the following cell to evaluate the performance of the forward pass running on the CPU:

In [14]:
%%timeit 
ans = fixed_model(x)

66 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


... and now the GPU:

In [15]:
%%timeit 
torch.cuda.synchronize() # Make sure there are no pending GPU computations
ans = fixed_model_gpu(x_gpu)        # Feed it through the model! 
torch.cuda.synchronize() # Make sure there are no pending GPU computations

633 µs ± 5.37 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


You should observe that even a simple forward pass like this is significantly faster on the GPU. So for the rest of the assignment (and when you go train your models in assignment 3 and your project!), you should use the GPU datatype for your model and your tensors: as a reminder that is *torch.cuda.FloatTensor* (in our notebook here as *gpu_dtype*)

### Train the model.

Now that you've seen how to define a model and do a single forward pass of some data through it, let's  walk through how you'd actually train one whole epoch over your training data (using the simple_model we provided above).

Make sure you understand how each PyTorch function used below corresponds to what you implemented in your custom neural network implementation.

Note that because we are not resetting the weights anywhere below, if you run the cell multiple times, you are effectively training multiple epochs (so your performance should improve).

First, set up an RMSprop optimizer (using a 1e-3 learning rate) and a cross-entropy loss function:

In [16]:
loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = optim.RMSprop(fixed_model_gpu.parameters(), lr=1e-3) 

In [17]:
%%time
# This sets the model in "training" mode. This is relevant for some layers that may have different behavior
# in training mode vs testing mode, such as Dropout and BatchNorm. 
fixed_model_gpu.train()

# Load one batch at a time.
for t, (x, y) in enumerate(loader_train):
    x = x.type(gpu_dtype)
    y = y.type(gpu_dtype).long()

    # This is the forward pass: predict the scores for each class, for each x in the batch.
    scores = fixed_model_gpu(x)
    
    # Use the correct y values and the predicted y values to compute the loss.
    loss = loss_fn(scores, y)
    
    if (t + 1) % print_every == 0:
        print('t = %d, loss = %.4f' % (t + 1, loss.data.item()))

    # Zero out all of the gradients for the variables which the optimizer will update.
    optimizer.zero_grad()
    
    # This is the backwards pass: compute the gradient of the loss with respect to each 
    # parameter of the model.
    loss.backward()
    
    # Actually update the parameters of the model using the gradients computed by the backwards pass.
    optimizer.step()

t = 100, loss = 1.4795
t = 200, loss = 1.5160
t = 300, loss = 1.4265
t = 400, loss = 1.2402
t = 500, loss = 1.2600
t = 600, loss = 1.3471
t = 700, loss = 1.2492
CPU times: user 7.5 s, sys: 200 ms, total: 7.7 s
Wall time: 7.74 s


In [18]:
%%time
# This sets the model in "training" mode. This is relevant for some layers that may have different behavior
# in training mode vs testing mode, such as Dropout and BatchNorm. 
fixed_model_gpu.train()

# Load one batch at a time.
for t, (x, y) in enumerate(loader_train):
    x_tensor = x.type(gpu_dtype)
    y_tensor = y.type(gpu_dtype).long()

    # This is the forward pass: predict the scores for each class, for each x in the batch.
    scores = fixed_model_gpu(x_tensor)
    
    # Use the correct y values and the predicted y values to compute the loss.
    loss = loss_fn(scores, y_tensor)
    
    if (t + 1) % print_every == 0:
        print('t = %d, loss = %.4f' % (t + 1, loss.data.item()))

    # Zero out all of the gradients for the variables which the optimizer will update.
    optimizer.zero_grad()
    
    # This is the backwards pass: compute the gradient of the loss with respect to each 
    # parameter of the model.
    loss.backward()
    
    # Actually update the parameters of the model using the gradients computed by the backwards pass.
    optimizer.step()

t = 100, loss = 1.0304
t = 200, loss = 1.0608
t = 300, loss = 1.0372
t = 400, loss = 0.8611
t = 500, loss = 0.9014
t = 600, loss = 0.9702
t = 700, loss = 0.9000
CPU times: user 7.44 s, sys: 200 ms, total: 7.64 s
Wall time: 7.67 s


Now you've seen how the training process works in PyTorch. To save you writing boilerplate code, we're providing the following helper functions to help you train for multiple epochs and check the accuracy of your model:

In [19]:
def train(model, loss_fn, optimizer, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, (x, y) in enumerate(loader_train):
            x_tensor = x.type(gpu_dtype)
            y_tensor = y.type(gpu_dtype).long()

            scores = model(x_tensor)
            
            loss = loss_fn(scores, y_tensor)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data.item()))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_tensor = x.type(gpu_dtype)

        scores = model(x_tensor)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return 100 * acc

### Check the accuracy of the model.

Let's see the train and check_accuracy code in action -- feel free to use these methods when evaluating the models you develop below.

You should get a training loss of around 1.2-1.4, and a validation accuracy of around 50-60%. As mentioned above, if you re-run the cells, you'll be training more epochs, so your performance will improve past these numbers.

But don't worry about getting these numbers better -- this was just practice before you tackle designing your own model.

In [20]:
%%time
torch.cuda.random.manual_seed(12345)
fixed_model_gpu.apply(reset)
train(fixed_model_gpu, loss_fn, optimizer, num_epochs=1)
check_accuracy(fixed_model_gpu, loader_val)

Starting epoch 1 / 1
t = 100, loss = 1.2813
t = 200, loss = 1.4695
t = 300, loss = 1.4219
t = 400, loss = 1.2472
t = 500, loss = 1.1583
t = 600, loss = 1.2737
t = 700, loss = 1.2667
Checking accuracy on validation set
Got 553 / 1000 correct (55.30)
CPU times: user 7.54 s, sys: 95.9 ms, total: 7.64 s
Wall time: 7.67 s


### Don't forget the validation set!

And note that you can use the check_accuracy function to evaluate on either the test set or the validation set, by passing either **loader_test** or **loader_val** as the second argument to check_accuracy. You should not touch the test set until you have finished your architecture and hyperparameter tuning, and only run the test set once at the end to report a final value. 

## Train a _great_ model on CIFAR-10!

Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves **>=70%** accuracy on the CIFAR-10 **validation** set. You can use the check_accuracy and train functions from above.

### Things you should try:
- **Filter size**: Above we used 7x7; this makes pretty pictures but smaller filters may be more efficient
- **Number of filters**: Above we used 32 filters. Do more or fewer do better?
- **Pooling vs Strided Convolution**: Do you use max pooling or just stride convolutions?
- **Batch normalization**: Try adding spatial batch normalization after convolution layers and vanilla batch normalization after affine layers. Do your networks train faster?
- **Network architecture**: The network above has two layers of trainable parameters. Can you do better with a deep network? Good architectures to try include:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Global Average Pooling**: Instead of flattening and then having multiple affine layers, perform convolutions until your image gets small (7x7 or so) and then perform an average pooling operation to get to a 1x1 image picture (1, 1 , Filter#), which is then reshaped into a (Filter#) vector. This is used in [Google's Inception Network](https://arxiv.org/abs/1512.00567) (See Table 1 for their architecture).
- **Regularization**: Add l2 weight regularization, or perhaps use Dropout.

### Tips for training
For each network architecture that you try, you should tune the learning rate and regularization strength. When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations
- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.
- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

### Going above and beyond
If you are feeling adventurous there are many other features you can implement to try and improve your performance. You are **not required** to implement any of these; however they would be good things to try for extra credit.

- Alternative update steps: For the assignment we implemented SGD+momentum, RMSprop, and Adam; you could try alternatives like AdaGrad or AdaDelta.
- Alternative activation functions such as leaky ReLU, parametric ReLU, ELU, or MaxOut.
- Model ensembles
- Data augmentation
- New Architectures
  - [ResNets](https://arxiv.org/abs/1512.03385) where the input from the previous layer is added to the output.
  - [DenseNets](https://arxiv.org/abs/1608.06993) where inputs into previous layers are concatenated together.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

If you do decide to implement something extra, clearly describe it in the "Extra Credit Description" cell below.

### What we expect
At the very least, you should be able to train a ConvNet that gets at least 70% accuracy on the validation set. This is just a lower bound - if you are careful it should be possible to get accuracies much higher than that! Extra credit points will be awarded for particularly high-scoring models or unique approaches.

You should use the space below to experiment and train your network. 

Have fun and happy training!

In [21]:
%%time
# Train your model here, and make sure the output of this cell is the accuracy of your best model on the 
# train, val, and test sets. Here's some code to get you started. The output of this cell should be the training
# and validation accuracy on your best model (measured by validation accuracy).

model = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1)),    # (N, 64, 32, 32)
            
                nn.BatchNorm2d(64),
                nn.ReLU(inplace = True),
                nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1)),  # (N, 128, 30, 30)
                nn.MaxPool2d(kernel_size=2, stride=2),                  # (N, 128, 15, 15)
    
                nn.BatchNorm2d(128),
                nn.ReLU(inplace = True),
                nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1)), # (N, 256, 13, 13)
                nn.MaxPool2d(kernel_size=2, stride=2),                  # (N, 256, 6, 6)
    
                nn.BatchNorm2d(256),
                nn.ReLU(inplace = True),
                nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1)), # (N, 256, 4, 4)
                nn.MaxPool2d(kernel_size=2, stride=2),                  # (N, 256, 2, 2)
            
                Flatten(),                                              # (N, 256 * 2 * 2)
    
                nn.Linear(1024, 256),                                   # (N, 256)
                nn.ReLU(inplace = True),
                nn.Linear(256, 10),                                     # (N, 10)
              ).type(gpu_dtype)

loss_fn = nn.CrossEntropyLoss().type(gpu_dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train(model, loss_fn, optimizer, num_epochs=1)
check_accuracy(model, loader_val)

Starting epoch 1 / 1
t = 100, loss = 1.3124
t = 200, loss = 1.2625
t = 300, loss = 1.2884
t = 400, loss = 0.8771
t = 500, loss = 0.9828
t = 600, loss = 0.8588
t = 700, loss = 1.0204
Checking accuracy on validation set
Got 691 / 1000 correct (69.10)
CPU times: user 9.8 s, sys: 128 ms, total: 9.93 s
Wall time: 9.96 s


Tuning learning rate

In [22]:
%%time
accuracies = []
learning_rates = np.random.uniform(1, 2, size=10) *  1e-3
for lr in tqdm(learning_rates):
    optimizer = optim.Adam(model.parameters(), lr = lr)

    train(model, loss_fn, optimizer, num_epochs=1)
    check_accuracy(model, loader_train)
    curr_acc = check_accuracy(model, loader_val)
    
    accuracies.append(curr_acc)

  0%|          | 0/10 [00:00<?, ?it/s]

Starting epoch 1 / 1
t = 100, loss = 0.7468
t = 200, loss = 0.9271
t = 300, loss = 0.9559
t = 400, loss = 0.7952
t = 500, loss = 0.8344
t = 600, loss = 0.5610
t = 700, loss = 0.8316
Checking accuracy on validation set


 10%|█         | 1/10 [00:18<02:42, 18.11s/it]

Got 35841 / 49000 correct (73.14)
Checking accuracy on validation set
Got 708 / 1000 correct (70.80)
Starting epoch 1 / 1
t = 100, loss = 0.6252
t = 200, loss = 0.6109
t = 300, loss = 0.6076
t = 400, loss = 0.5689
t = 500, loss = 0.6175
t = 600, loss = 0.3696
t = 700, loss = 0.6969
Checking accuracy on validation set


 20%|██        | 2/10 [00:35<02:24, 18.03s/it]

...

Got 45712 / 49000 correct (93.29)
Checking accuracy on validation set
Got 795 / 1000 correct (79.50)
Starting epoch 1 / 1
t = 100, loss = 0.0355
t = 200, loss = 0.1665
t = 300, loss = 0.2204
t = 400, loss = 0.0368
t = 500, loss = 0.0764
t = 600, loss = 0.3150
t = 700, loss = 0.0614
Checking accuracy on validation set


 90%|█████████ | 9/10 [02:41<00:17, 17.96s/it]

Got 47534 / 49000 correct (97.01)
Checking accuracy on validation set
Got 800 / 1000 correct (80.00)
Starting epoch 1 / 1
t = 100, loss = 0.0861
t = 200, loss = 0.0464
t = 300, loss = 0.0140
t = 400, loss = 0.0564
t = 500, loss = 0.0220
t = 600, loss = 0.0857
t = 700, loss = 0.2844
Checking accuracy on validation set


100%|██████████| 10/10 [02:59<00:00, 17.99s/it]

Got 47646 / 49000 correct (97.24)
Checking accuracy on validation set
Got 799 / 1000 correct (79.90)
CPU times: user 2min 52s, sys: 6.54 s, total: 2min 59s
Wall time: 2min 59s


In [23]:
"Best accuracy: {}, best learning rate: {}".format(max(accuracies), learning_rates[np.argmax(accuracies)])

'Best accuracy: 80.0, best learning rate: 0.0010758497016643664'

More epochs

In [24]:
%%time
optimizer = optim.Adam(model.parameters(), lr=learning_rates[np.argmax(accuracies)])
train(model, loss_fn, optimizer, num_epochs=5)
check_accuracy(model, loader_val)

Starting epoch 1 / 5
t = 100, loss = 0.0814
t = 200, loss = 0.0171
t = 300, loss = 0.0534
t = 400, loss = 0.1235
t = 500, loss = 0.1726
t = 600, loss = 0.0246
t = 700, loss = 0.0612
Starting epoch 2 / 5
t = 100, loss = 0.0440
t = 200, loss = 0.0747
t = 300, loss = 0.0336
t = 400, loss = 0.0146
t = 500, loss = 0.0455
t = 600, loss = 0.0047
t = 700, loss = 0.0164
Starting epoch 3 / 5
t = 100, loss = 0.0178
t = 200, loss = 0.0077
t = 300, loss = 0.0599
t = 400, loss = 0.0181
t = 500, loss = 0.0067
t = 600, loss = 0.0359
t = 700, loss = 0.1036
Starting epoch 4 / 5
t = 100, loss = 0.0064
t = 200, loss = 0.0416
t = 300, loss = 0.0210
t = 400, loss = 0.0429
t = 500, loss = 0.1143
t = 600, loss = 0.0267
t = 700, loss = 0.0336
Starting epoch 5 / 5
t = 100, loss = 0.0029
t = 200, loss = 0.0384
t = 300, loss = 0.0030
t = 400, loss = 0.0911
t = 500, loss = 0.0192
t = 600, loss = 0.0019
t = 700, loss = 0.0238
Checking accuracy on validation set
Got 822 / 1000 correct (82.20)
CPU times: user 48.2 s,

### Describe what you did 

In the cell below you should write an explanation of what you did, any additional features that you implemented, and any visualizations or graphs that you make in the process of training and evaluating your network.

Tell us here!

*I started with a first Conv layer and then a sandwitch of 3 (BatchNorm -> ReLU -> Conv -> Maxpool) blocks.
I didn't find adding dropout to be helpful, maybe because of the presence of BatchNorm.*

## Test set -- run this only once

Now that we've gotten a result we're happy with, we test our final model on the test set (which you should store in best_model).  This would be the score we would achieve on a competition. Think about how this compares to your validation set accuracy.

In [25]:
best_model = model
check_accuracy(best_model, loader_test)

Checking accuracy on test set
Got 7969 / 10000 correct (79.69)


79.69000000000001

## Going further with PyTorch

The next assignment will make heavy use of PyTorch. You might also find it useful for your projects. 

Here's a nice tutorial by Justin Johnson that shows off some of PyTorch's features, like dynamic graphs and custom NN modules: http://pytorch.org/tutorials/beginner/pytorch_with_examples.html

If you're interested in reinforcement learning for your final project, this is a good (more advanced) DQN tutorial in PyTorch: http://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html